In [613]:
!pip install spotipy
!pip install geopandas
!pip install geopy
!pip install requests
!pip install lxml
!pip install pandas
!pip install numpy
!pip install gmplot

In [614]:
import requests
import spotipy
import geopandas
import geopy
from spotipy.oauth2 import SpotifyClientCredentials 
import requests
from lxml import html
import re
import pandas as pd
import numpy as np
import math
from geopy.geocoders import Nominatim

## Spotify Scraper

### Connect to Spotify API

In [615]:
client_id = '693faf0bce4f4fb0aa5fc5d8b3cc0b25'
client_secret = 'c2ab38a172f44774a0cfae1d7734130f'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 

### Scraper Functions
get_artistID_by_artistName - check for reliability, better search

In [616]:
#not deemed reliable yet
def get_artistID_by_artistName(artistName):
        artist = sp.search(q=artistName, type="artist")
        #print(artist)
        artistObject = artist['artists']['items']
        if len(artistObject) == 0:
            return "NOT IN SPOTIFY"
        else:
            artistID = artistObject[0]['id']
            #print('AID: ', artistID)
            return artistID;

def get_genre(artistID):
    #navigate to artist
    artist = sp.artist(artistID)
    #Obtain genre
    genres = artist['genres']
    if len(genres) == 0:
        return 'NONE'
    else:
        return genres

def get_followers(artistID):
    #navigate to artist
    artist = sp.artist(artistID)
    #Obtain follower count
    followers = int(artist['followers']['total'])
    return followers

def get_monthlyListeners(artistID):
    r = requests.get('https://open.spotify.com/artist/{}'.format(artistID), timeout=10)
    match = re.search(r'Monthly Listeners', r.text)
    print(r.text)
    #get monthly listeners starting point
    string = r.text[match.start():]
    #split at colon
    strings = string.split(':')
    strings1 = strings[1].split(',')
    monthlyListeners = strings1[0].split(",")
    #grab second string
    return int(monthlyListeners[0].lstrip())

def get_cities(artistID):
    r = requests.get('https://open.spotify.com/artist/{}'.format(artistID), timeout=10)
    match = re.search(r'Where People Listen', r.text)
    #get where people listen starting point
    string = r.text[match.start():]
    #split at colon
    strings = string.split(':')
    strings1 = strings[1].split('"')
    cities = strings1[0].split(",")
    #grab second string
    return cities

## Songkick Scraper

### Connect to Songkick API

In [617]:
# Define API Key
api_key = "FYSeGSNlg6HXIfAr"

### Scraper Functions

get_venueArtists: add in festival/lineup capabilities

get_venueGenres: clean up mapping

get_venueSize: Decide how to use followers/listeners, fix metric (within the 30-60 percentile?), regression using Songkick Venue Capacity?

In [618]:
def get_venueArtists(metroID):
    # request json
    r = requests.get("https://api.songkick.com/api/3.0/metro_areas/"+ str(metroID) +"/calendar.json?apikey=" + str(api_key))
    # parse for events
    events = r.json()["resultsPage"]["results"]["event"]
    # define dictionary for storage
    venue_artist_dict = {}
    #loop through events
    for i in range(len(events)):
        # The artist
        if events[i]["performance"] != []:
            artist = events[i]["performance"][0]["artist"]["displayName"]
        else:
            artist = events[i]["displayName"]
        # Venue Name
        venue = events[i]["venue"]["displayName"]
        venueID = events[i]["venue"]["id"]
       # venueLat = events[i]["venue"]["lat"]
        # venueLong = events[i]["venue"]["lng"]
        # if the venue is not already in dict, add it with the artist as its value
        if venue not in venue_artist_dict.keys():
            venue_artist_dict[venue] = [artist]
        # if it is in the dict, add the act to its 
        else:
            if artist not in venue_artist_dict[venue]:
                venue_artist_dict[venue].append(artist)
    return venue_artist_dict

def get_venueGenres(venue_artist_dict):
    venue_genre_dict = {}
    # loop through venues
    for venue in venue_artist_dict.keys():
        # loop through artists at each venue
        genre_list = []
        for artist in venue_artist_dict[venue]:
            #get artist id
            id_return = get_artistID_by_artistName(artist)
            # if its too small of an artist, ignore it
            if id_return == 'NOT IN SPOTIFY':
                artist_genre = 'NONE'
                # get the genre of artists in spotify
            else:
                artist_genre = get_genre(id_return)
                print
                mapped_genre=-1
                # replace the genre with a code... NEED TO CLEAN UP
                if any("funk" in s for s in artist_genre):
                    mapped_genre = 0
                if any("rock" in s for s in artist_genre):
                    mapped_genre = 1 
                if any("punk" in s for s in artist_genre):
                    mapped_genre = 1
                if any("reggae" in s for s in artist_genre):
                    mapped_genre = 2
                if any("r&b" in s for s in artist_genre):
                    mapped_genre = 3
                if any("soul" in s for s in artist_genre):
                    mapped_genre = 3
                if any("rap" in s for s in artist_genre):
                    mapped_genre = 4
                if any("bounce" in s for s in artist_genre):
                    mapped_genre = 4
                if any("pop" in s for s in artist_genre):
                    mapped_genre = 5
                if any("jazz" in s for s in artist_genre):
                    mapped_genre = 6
                if any("electronic" in s for s in artist_genre):
                    mapped_genre = 7
                if any("indie" in s for s in artist_genre):
                    mapped_genre = 8
                if any("country" in s for s in artist_genre):
                    mapped_genre = 9
                if any("cajun" in s for s in artist_genre):
                    mapped_genre = 9
                # add genre to list
                genre_list.append(mapped_genre)
        # assign genres to venue
        venue_genre_dict[venue] = list(set([x for x in genre_list if x != -1]))
    # return dictionary
    return venue_genre_dict

def get_venueSize(venue_artist_dict):
    venue_size_dict = {}
    # loop through venues
    for venue in venue_artist_dict.keys():
        # loop through artists at each venue
        size_list = []
        for artist in venue_artist_dict[venue]:
            #get artist id
            id_return = get_artistID_by_artistName(artist)
            # if its too small of an artist, its size is basically 0
            if id_return == 'NOT IN SPOTIFY':
                size_list.append(0)
            # get their followers
            else:
                artist_followers = get_followers(id_return)
                size_list.append(artist_followers)
        venue_size_dict[venue] = np.mean(np.array(size_list))
    return venue_size_dict

In [619]:
#get requested city id
#example


cityIDs = []
def get_city_id(locations):
    cityIDs = []
    for i in range(len(locations)):
        r = requests.get("https://api.songkick.com/api/3.0/search/locations.json?query=" + locations[i] + "&apikey=" + str(api_key))
        cityID = r.json()["resultsPage"]["results"]["location"][0]["metroArea"]["id"]
        cityIDs.append(cityID)
    return cityIDs


## Matching Functions
Both: Need to figure out a way to clean it up


In [620]:
#
# FUNCTION TO ADD ELEMENTS TO DICT
#
def append_value(dict_obj, key, value):
    if key in dict_obj:
        if not isinstance(dict_obj[key], tuple):
            dict_obj[key] = [dict_obj[key]]
        dict_obj[key].append(value)
    else:
        dict_obj[key] = value

#
# GENRE MATCH FUNCTION
# if artist genre identity is a match for venue genre, match (hard constraints)
#

def genre_match(d, genre_of):
    #print('in genre check function')
    genre_filtered_dictionary = {}

    # iterate through artist's given constraints O(n)
    for i in genre_of:
        for key, value in d.items():
            for j in value:
                if i == j:
                    append_value(genre_filtered_dictionary, key, value)
                else:
                    pass

    return genre_filtered_dictionary

def size_match(d,size_of):
    perfect_match_dict={}
    for key,value in d.items():
        num=value
        difference=abs(num-size_of)
        per=difference/size_of
        percentage1=1-per
        if(percentage1>=.4):
            perfect_match_dict[key]= round(percentage1, 2)
            None
    return perfect_match_dict

## New Orleans Example

In [621]:
#use names from class... THIS PART WILL BE CHANGED!

names = ["Taylor Swift","Vampire Weekend","The Doobie Brothers", "Greta Van Fleet", "Tank and the Bangas", "Lady Gaga"]
def create_df(names):
    df = pd.DataFrame({"band_name": names})
    df["id"] = df.band_name.apply(get_artistID_by_artistName)
    df["genre"] = df.id.apply(get_genre)
    #df["listeners"] = df.id.apply(get_monthlyListeners)
    df["followers"] = df.id.apply(get_followers)
    #pd.set_option("display.max_rows", None, "display.max_columns", None, 'display.max_colwidth',None)
    df['mapped_genre'] = [[] for _ in range(df.shape[0])]

    for i in range(df.genre.size):
        if any("funk" in s for s in df.genre[i]):
            df['mapped_genre'][i].append('0')

        if any("rock" in s for s in df.genre[i]):
            df['mapped_genre'][i].append('1')

        if any("punk" in s for s in df.genre[i]):
            df['mapped_genre'][i].append('1')

        if any("reggae" in s for s in df.genre[i]):
            df['mapped_genre'][i].append('2')

        if any("r&b" in s for s in df.genre[i]):
            df['mapped_genre'][i].append('3')

        if any("rap" in s for s in df.genre[i]):
            df['mapped_genre'][i].append('4')

        if any("pop" in s for s in df.genre[i]):
            df['mapped_genre'][i].append('5')

        if any("jazz" in s for s in df.genre[i]):
            df['mapped_genre'][i].append('6')

        if any("electronic" in s for s in df.genre[i]):
            df['mapped_genre'][i].append('7')

        if any("indie" in s for s in df.genre[i]):
            df['mapped_genre'][i].append('8')

        if any("country" in s for s in df.genre[i]):
            df['mapped_genre'][i].append('9')
    return df
df = create_df(names)

In [622]:
#
# OUTPUT SECTION
# change this portion when looking at different bands
#
cityID = "11772"
def output_matching_df(cityID, df):
    city = get_venueArtists(cityID)
    city_venue_genres = get_venueGenres(city)
    city_venue_sizes = get_venueSize(city)
    column_names = ["Artist", "Artist Genre", "Artist Size", "Genre Matches", "Size Matches", "Set"]
    output_df = pd.DataFrame(columns = column_names)

    for i in range(0,len(df)):
        #get artist genres
        artist_genre = []
        for j in range(0,len(df.mapped_genre[i])):
            artist_genre.append(int(df.mapped_genre[i][j]))

        #get artist size
        artist_size = df.followers[i]

        #calculate genre matches
        genre_matches = genre_match(city_venue_genres ,artist_genre)


        #calculate size matches
        size_matches = size_match(city_venue_sizes ,artist_size)
        output_df = output_df.append({'Artist':df.band_name[i],'Artist Genre':artist_genre,
                                      'Artist Size':artist_size,'Genre Matches':genre_matches,
                                      'Size Matches':size_matches}, ignore_index=True)

    return output_df
output_matching_df(cityID, df)

,Artist,Artist Genre,Artist Size,Genre Matches,Size Matches,Set
0,Taylor Swift,[5],38167131,{},{},NaN
1,Vampire Weekend,"[1, 5, 8]",1703858,"{'Smoothie King Center': [1], 'Gasa Gasa': [8]...",{'House of Blues - New Orleans presented by Cr...,NaN
2,The Doobie Brothers,"[1, 9]",1507523,"{'Smoothie King Center': [1], 'Music Box Villa...",{'House of Blues - New Orleans presented by Cr...,NaN
3,Greta Van Fleet,[1],1390953,{'Smoothie King Center': [1]},{'House of Blues - New Orleans presented by Cr...,NaN
4,Tank and the Bangas,"[0, 8]",159625,"{'The Howlin' Wolf': [0, 6], 'Broadside': [0],...",{'Music Box Village': 0.96},NaN
5,Lady Gaga,[5],16776635,{},{},NaN


### Traveling Salesperson Problem/Algorithm (TSP) - Example with Tank and the Bangas

#### Get latitude and longitude of each venue recommend by genre match

In [623]:
value = output_matching_df(cityID, df).iloc[4]['Genre Matches']
value

{"The Howlin' Wolf": [0, 6],
 'Broadside': [0],
 'Joy Theater': [0],
 'Gasa Gasa': [8],
 "Tipitina's": [8]}

In [624]:
locations = ["New Orleans, LA"]
def get_latLong(locations):
    column_names = ["Venue", "Latitude", "Longitude"]
    venue_info_df = pd.DataFrame(columns = column_names)
    for i in range(len(locations)):
        for key in value.keys():
            print(key)
            r = requests.get("https://api.songkick.com/api/3.0/search/venues.json?query={"+ key + "}&apikey=" + str(api_key))
            venueLat = r.json()['resultsPage']['results']['venue'][0]['lat']
            venueLng = r.json()['resultsPage']['results']['venue'][0]['lng']
            #not including venues that have no data for now
            if(venueLat == None):
                venueLat = None
            else:
                venue_info_df = venue_info_df.append({'Venue':key,'Latitude':venueLat,'Longitude':venueLng}, ignore_index=True)
        venue_info_df = venue_info_df.set_index("Venue")
        return venue_info_df
venue_info_df = get_latLong(locations)
venue_info_df

The Howlin' Wolf
Broadside
Joy Theater
Gasa Gasa
Tipitina's


,Latitude,Longitude
Venue,,
The Howlin' Wolf,29.94272,-90.066060
Broadside,29.96916,-90.083560
Joy Theater,29.95601,-90.073980
Gasa Gasa,29.93494,-90.107852
Tipitina's,29.91741,-90.100780


#### Get Lat and Long of Starting Point

In [625]:
#not being used right now
#starting point right now = first item in list
startingPoint = "4920 Freret St, New Orleans, LA 70115"
locator = Nominatim(user_agent="myGeocoder")
location = locator.geocode(startingPoint)
startingLat = location.latitude
startingLng =location.longitude

#### Function to calculate distance between two venues based on lat and long

In [626]:

def distance_on_earth(lat1, long1, lat2, long2, radius=6378.388):
    """
    Compute distance between two points on earth specified by latitude/longitude.
    The earth is assumed to be a perfect sphere of given radius. The radius defaults
    to 6378.388 kilometers. To convert to miles, divide by 1.60934
    
    Reference
    ---------
    Adopted from John D. Cook's blog post: 
    http://www.johndcook.com/blog/python_longitude_latitude/
    """
    # Convert latitude and longitude to spherical coordinates in radians.
    degrees_to_radians = np.pi / 180.0

    # phi = 90 - latitude
    phi1 = (90.0 - lat1) * degrees_to_radians
    phi2 = (90.0 - lat2) * degrees_to_radians

    # theta = longitude
    theta1 = long1 * degrees_to_radians
    theta2 = long2 * degrees_to_radians
         
    # Compute spherical distance from spherical coordinates.     
    cos = (np.sin(phi1) * np.sin(phi2)* np.cos(theta1 - theta2) + 
           np.cos(phi1) * np.cos(phi2))
    arc = np.arccos(cos)
    rv = arc * radius
    return rv

#### Get distances between each venue and create distance matrix

In [627]:
mile_in_km = 1.60934
distances = {}
for i in range(len(venue_info_df.index)):
    a = venue_info_df.index[i]
    distances[a] = {}
    for j in range(len(venue_info_df.index)):
        b = venue_info_df.index[j]
        if j == i:
            distances[a][b] = 0.
        elif j > i:
            distances[a][b] = distance_on_earth(venue_info_df['Latitude'][a],
                                                venue_info_df['Longitude'][a],
                                                venue_info_df['Latitude'][b],
                                                venue_info_df['Longitude'][b])
        else:
            distances[a][b] = distances[b][a]
distances = (pd.DataFrame(distances)/mile_in_km)

In [628]:
distances

,The Howlin' Wolf,Broadside,Joy Theater,Gasa Gasa,Tipitina's
The Howlin' Wolf,0.000000,2.108338,1.034641,2.562274,2.719833
Broadside,2.108338,0.000000,1.075659,2.779019,3.725571
Joy Theater,1.034641,1.075659,0.000000,2.499222,3.116136
Gasa Gasa,2.562274,2.779019,2.499222,0.000000,1.284595
Tipitina's,2.719833,3.725571,3.116136,1.284595,0.000000


In [629]:
!pip install python-tsp

#### Use python-tsp library to solve tsp with dynamic programming

In [630]:
import numpy as np
from python_tsp.distances import great_circle_distance_matrix
from python_tsp.exact import solve_tsp_dynamic_programming

arr = []
for i in range(len(venue_info_df.index)):
    arr.append([venue_info_df['Latitude'][venue_info_df.index][i],venue_info_df['Longitude'][venue_info_df.index][i]])
sources = np.array(arr)

In [631]:
distance_matrix = great_circle_distance_matrix(sources)

In [632]:
distance_matrix

array([[   0.        , 3389.10198265, 1663.16122819, 4118.79321796,
        4372.06550847],
       [3389.10198265,    0.        , 1729.09606106, 4467.20550825,
        5988.76594272],
       [1663.16122819, 1729.09606106,    0.        , 4017.43856891,
        5009.11436569],
       [4118.79321796, 4467.20550825, 4017.43856891,    0.        ,
        2064.95557159],
       [4372.06550847, 5988.76594272, 5009.11436569, 2064.9555716 ,
           0.        ]])

In [633]:
permutation, distance = solve_tsp_dynamic_programming(distance_matrix)
distance

14296.483877566476

In [634]:
#convert distance from km to miles
distance = distance/1609
distance

8.885322484503714

In [635]:
permutation

[0, 2, 1, 3, 4]

In [636]:
s = pd.Series(permutation)

In [637]:
mappingDict = {}
for i in range(len(s)):
    mappingDict[i] = venue_info_df.index[i]


In [638]:
s = s.map(mappingDict)

In [639]:
optimalPath = s.tolist()

### Men I Trust Optimal Tour Path in New Orleans

##### optimal path

In [640]:
optimalPath

["The Howlin' Wolf", 'Joy Theater', 'Broadside', 'Gasa Gasa', "Tipitina's"]

##### total distance in miles

In [641]:
distance

8.885322484503714

#### Function to call entire tour with option of multiple cities

In [642]:
###FOLLOWING NEED TO BE INPUTTED BY USER
#locations = [] can be one item or multiple
#names = [] can be one item or multiple


In [645]:
names = ["Vampire Weekend"]
locations = ["New York City, New York", "Los Angeles, CA", "Nashville, TN"]
df = create_df(names)
df

,band_name,id,genre,followers,mapped_genre
0,Vampire Weekend,5BvJzeQpmsdsFp4HGUYUEx,"[baroque pop, indie pop, indie rock, modern ro...",1703858,"[1, 5, 8]"


In [647]:
cities = get_city_id(locations)
cities
for cityID in cities:
    print(cityID)
    display(output_matching_df(cityID,df))
    value = output_matching_df(cityID, df).iloc[0]['Genre Matches']
    venue_info_df = get_latLong(locations)
    display(venue_info_df)
    arr = []
    for i in range(len(venue_info_df.index)):
        arr.append([venue_info_df['Latitude'][venue_info_df.index][i],venue_info_df['Longitude'][venue_info_df.index][i]])
    sources = np.array(arr)
    distance_matrix = great_circle_distance_matrix(sources)
    permutation, distance = solve_tsp_dynamic_programming(distance_matrix)
    distance = distance/1609
    permutation
    s = pd.Series(permutation)
    mappingDict = {}
    for i in range(len(s)):
        mappingDict[i] = venue_info_df.index[i]
    s = s.map(mappingDict)
    optimalPath = s.tolist()
    print(optimalPath)


    

7644


,Artist,Artist Genre,Artist Size,Genre Matches,Size Matches,Set
0,Vampire Weekend,"[1, 5, 8]",1703858,"{'Brooklyn Bowl': [1], 'Regina's': [5], 'The H...",{},NaN


Brooklyn Bowl
Regina's
The Hall, Elsewhere
Webster Hall
City Winery
The Vogel, Count Basie Center for the Arts
Music Hall of Williamsburg
Brooklyn Steel


,Latitude,Longitude
Venue,,
Brooklyn Bowl,40.721880,-73.957390
Regina's,50.452155,-104.606591
"The Hall, Elsewhere",40.709340,-73.923230
Webster Hall,40.731770,-73.989110
City Winery,40.743560,-74.010470
"The Vogel, Count Basie Center for the Arts",40.347050,-74.064310
Music Hall of Williamsburg,40.719283,-73.961710
Brooklyn Steel,40.719360,-73.938780


['Brooklyn Bowl', 'Brooklyn Steel', 'The Hall, Elsewhere', 'The Vogel, Count Basie Center for the Arts', "Regina's", 'City Winery', 'Webster Hall', 'Music Hall of Williamsburg']
17835


...

['City National Grove of Anaheim', 'The Wiltern', 'The Echo', 'The Forum', 'NOS Events Center', 'University of Riverside', 'Riverside Municipal Auditorium', "A Stone's Throw Winery"]
11104


,Artist,Artist Genre,Artist Size,Genre Matches,Size Matches,Set
0,Vampire Weekend,"[1, 5, 8]",1703858,"{'Ridenour Rehearsal Studios': [1], 'The Basem...",{},NaN


Ridenour Rehearsal Studios
The Basement East
Exit In
Ryman Auditorium


,Latitude,Longitude
Venue,,
Ridenour Rehearsal Studios,35.83086,-86.39678
The Basement East,36.17564,-86.75565
Exit In,36.15133,-86.80426
Ryman Auditorium,36.16117,-86.77869


['Ridenour Rehearsal Studios', 'The Basement East', 'Ryman Auditorium', 'Exit In']


In [ ]:
r.json()

In [651]:
r = requests.get("https://api.songkick.com/api/3.0/metro_areas/"+ "35129" +"/calendar.json?apikey=" + str(api_key))
events = r.json()["resultsPage"]["results"]["event"]
venue = events[i]["venue"]["id"]

venue

3759979